## PyCitySchools<a id='top'></a>

Every student's math and reading scores, as well as various information on the schools they attend, are provided. We aggregate the data to find obvious trends in school performance.

### Load data

* [Load data](#load_data) - the district-wide standardized test results

### Analysis

1. [District Summary](#District_Summary)

2. [School Summary](#School_Summary)

3. [Top Performing Schools](#Top_Performing_Schools)

4. [Bottom Performing Schools](#Bottom_Performing_Schools)

5. [Math Scores by Grade](#Math_Scores)

6. [Reading Scores by Grade](#Reading_Scores)

7. [Scores by School Spending](#School_Spending)

8. [Scores by School Size](#School_Size)

9. [Scores by School Type](#School_Type)

### Trends

* [Observations in school performance](#trends)


In [15]:
# Dependencies and Setup
import pandas as pd

## Load Data<a id='load_data'></a>

[To Top](#top)

In [16]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_data_to_load)
student_df = pd.read_csv(student_data_to_load)

In [17]:
st_df = student_df.copy()
sc_df = school_df.copy()
sc_df.sort_values(by='school_name', ignore_index=True, inplace=True)
sc_df.head()

,School ID,school_name,type,size,budget
0,7,Bailey High School,District,4976,3124928
1,6,Cabrera High School,Charter,1858,1081356
2,1,Figueroa High School,District,2949,1884411
3,13,Ford High School,District,2739,1763916
4,4,Griffin High School,Charter,1468,917500


In [18]:
st_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## Analysis

In [19]:
#-----------------------------------
# Display_DF(df)
#   returns a dataframe with the formated strings 
#   with '$' or '%' or float format. 
#-----------------------------------
def Display_DF(df, index_name=True):
    
    df2 = df.copy()
    for col in df2.columns:
        
        if "Budget" in col:
            df2[col]=df[col].map('${:,.2f}'.format)

        elif "%" in col:
            df2[col]=df[col].map('{:,.2f}%'.format)

        elif df[col].dtype==float:
            df2[col]=df[col].map('{:,.2f}'.format)
            
        elif df[col].dtype==int:
            df2[col]=df[col].map('{:,}'.format)
 
    if index_name==False:
        df2.index.name = None
        
    return df2

#-----------------------------------
# Score_Summary(df)
#
#   - Arguments : df   
#      df is a dataframe with 'Student ID', 'math_score' and 'reading_score'
#      
#   - Returns a dataframe with the averages of math/reading scores
#        and the percentage of the students passing math/reading/both
#-----------------------------------
def Score_Summary(df):
    
    total_students = df['Student ID'].count()
        
    avg_math    = df['math_score'].mean()
    avg_read    = df['reading_score'].mean()
    pass_math = df.loc[df['math_score']>=70,'Student ID'].count()
    pass_read = df.loc[df['reading_score']>=70,'Student ID'].count()
    pass_both = df.loc[(df['math_score']>=70) & (df['reading_score']>=70),'Student ID'].count()
    
    summary = pd.Series({
        'Average Math Score'    : avg_math,
        'Average Reading Score' : avg_read,
        '% Passing Math'        : (pass_math/total_students)*100,
        '% Passing Reading'     : (pass_read/total_students)*100,
        '% Overall Passing'     : (pass_both/total_students)*100})
    return summary

### 1. District Summary<a id='District_Summary'></a>

##### Schools

* the total number of schools

* the total budget

##### Students

* the total number of students

* the average math score 

* the average reading score

* the percentage of students with a passing math score (70 or greater)

* the percentage of students with a passing reading score (70 or greater)

* the percentage of students who passed math **and** reading (% Overall Passing)

[To Top](#top)

In [20]:
# Calculate the school summary
total_schools = sc_df['school_name'].count()
total_budget = sc_df['budget'].sum()

# Calculate the student summary
total_students = st_df['Student ID'].count()
student_score = Score_Summary(st_df).to_frame().transpose()

# Create district summary by combining the school and student summary results
district_summary = pd.DataFrame({
    'Total Schools' : total_schools,
    'Total Students': total_students,
    'Total Budget'  : total_budget}, index=[0])

district_summary = pd.concat([district_summary,student_score], axis=1)

# Print the district summary
Display_DF(district_summary)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


### 2. School Summary<a id='School_Summary'></a>

Summary of the key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
  [To Top](#top)

In [21]:
# Group the student data by schools
grouped_by_school = st_df.groupby(['school_name'])

# Calculate the score summary about each school
score_summary_sc = grouped_by_school.apply(Score_Summary)

# Calculate the 'Per Student Budget' 
# using 'buget' and 'size'(the number of students in the school)
sc_df['Per Student Budget'] = sc_df['budget'].div(sc_df['size'])

# Merge the school data and the score summary about each school
combined_df = pd.merge(sc_df, score_summary_sc, how="left", on="school_name")

# Set and organize the school summary
school_summary = combined_df.rename(columns={
        'type'   : 'School Type',
        'size'   : 'Total Students',
        'budget' : 'Total School Budget'})
school_summary.drop(columns=['School ID'], inplace=True)

school_summary.set_index('school_name', inplace=True)

# Print the dataframe
Display_DF(school_summary, index_name=False)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


### 3. Top Performing Schools <a id='Top_Performing_Schools'></a>

* Top five performing schools by % overall passing. ([To Top](#top))

In [22]:
# Print the sorted school summary by the overall passing percentage
top_perform_sc = school_summary.sort_values(by='% Overall Passing', ascending=False)
Display_DF(top_perform_sc, index_name=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


### 4. Bottom Performing Schools <a id='Bottom_Performing_Schools'></a>

* Five worst-performing schools by % overall passing.   [To Top](#top)

In [23]:
# Print the sorted school summary by the overall passing percentage
botton_perform_sc = school_summary.sort_values(by='% Overall Passing')
Display_DF(botton_perform_sc, index_name=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


### 5. Math Scores by Grade<a id='Math_Scores'></a>

* The average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.  [To Top](#top)

In [24]:
# Average scores of the groups by grade(first) and school name(second)
grade_score = st_df.groupby(['grade', 'school_name'])['math_score'].mean()

math_summary = pd.DataFrame({
    '9th' : grade_score['9th'],
    '10th': grade_score['10th'],
    '11th': grade_score['11th'],
    '12th': grade_score['12th']})

Display_DF(math_summary, index_name=False)

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


### 6. Reading Scores by Grade<a id='Reading_Scores'></a>
                                 
* The average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.  [To Top](#top)

In [25]:
# Average scores of the groups by grade(first) and school name(second)
grade_score = st_df.groupby(['grade', 'school_name'])['reading_score'].mean()

reading_summary = pd.DataFrame({
    '9th' : grade_score['9th'],
    '10th': grade_score['10th'],
    '11th': grade_score['11th'],
    '12th': grade_score['12th']})

Display_DF(reading_summary, index_name=False)

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


### 7. Scores by School Spending<a id='School_Spending'></a>

* School performances based on average Spending Ranges (Per Student):

  * Average Math Score
  
  * Average Reading Score
  
  * % Passing Math
  
  * % Passing Reading
  
  * Overall Passing Rate
  
  [To Top](#top)

In [26]:
# Set the bins with the Spending Ranges
bins_label = ['<$584', '$585-629', '$630-644', '$645-675']
bins_budget = [0, 584, 630, 645, 676]

# Binning along the ranges
df = school_summary.copy()
col_name = "Spending Ranges (Per Student)"
df[col_name]=pd.cut(df['Per Student Budget'], bins=bins_budget, labels=bins_label)

# Calculate the average of the groups
school_spending = df.groupby(col_name).mean()

# Print only the score related columns
Display_DF(school_spending[['Average Math Score', 'Average Reading Score',
                           '% Passing Math', '% Passing Reading',
                           '% Overall Passing']])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46%,96.61%,90.37%
$585-629,81.90,83.16,87.13%,92.72%,81.42%
$630-644,78.52,81.62,73.48%,84.39%,62.86%
$645-675,77.00,81.03,66.16%,81.13%,53.53%


### 8. Scores by School Size<a id='School_Size'></a>

* School performances based on school size:

  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate
  
  [To Top](#top)

In [27]:
# Set the bins with the School Size
bins_label = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
bins_size = [0,999,1999,5000]

# Binning along the size ranges
df = school_summary.copy()
df['School Size']=pd.cut(df['Total Students'], bins=bins_size, labels=bins_label)

# Calculate the average of the groups
school_size = df.groupby('School Size').mean()

# Print only the score related columns
Display_DF(school_size[['Average Math Score', 'Average Reading Score',
                           '% Passing Math', '% Passing Reading',
                           '% Overall Passing']])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


### 9. Scores by School Type<a id='School_Type'></a>

* School performances based on school type:

  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate
  [To Top](#top)

In [28]:
# Calculate the average of the groups by School Type
school_type = school_summary.groupby('School Type').mean()

# Print only the score related columns
Display_DF(school_type[['Average Math Score', 'Average Reading Score',
                           '% Passing Math', '% Passing Reading',
                           '% Overall Passing']])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%


## Observation<a id='trends'></a>

* The Charter schools perform better than the District schools in terms of the overall passing rate. ([Table](#School_Type))

    * All top five are Charter schools([Table](#Top_Performing_Schools)) and all bottom five are District schools([Table](#Bottom_Performing_Schools)).
    
* While the Charter schools maintain high passing rates for both subjects, the District schools have much less passing rate in Math. ([Table](#School_Type))

* The schools who spend more for each student perform worse. ([Table](#School_Spending))

* The midium sized schools perform the best. ([Table](#School_Size))

 [To Top](#top)